In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#data = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/DS/UNSW_NB15_training-set.csv')
data = pd.read_csv('../datasets/UNSW_NB15_training-set.csv')

# 10 categories

In [4]:
train_other = data[['spkts','dpkts','sbytes','dbytes','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm']]
train_protos = data[['proto']]
train_cats = data[['attack_cat']]

In [5]:
regex = r'^(?!(\barp\b|\bigmp\b|\bospf\b|\btcp\b|\budp\b)).*$'
train_protos = train_protos.replace(to_replace=regex, value='other', regex=True)
train_protos.describe()

,proto
count,82332
unique,6
top,tcp
freq,43095


In [6]:
train_dummies = pd.get_dummies(train_protos)
train_samples = pd.concat([train_other, train_dummies], axis=1)
train_samples.head()

,spkts,dpkts,sbytes,dbytes,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,proto_arp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
0,2,0,496,0,2,2,1,1,1,1,2,0,0,0,0,0,1
1,2,0,1762,0,2,2,1,1,1,1,2,0,0,0,0,0,1
2,2,0,1068,0,3,3,1,1,1,1,3,0,0,0,0,0,1
3,2,0,900,0,3,3,2,2,2,1,3,0,0,0,0,0,1
4,2,0,2126,0,3,3,2,2,2,1,3,0,0,0,0,0,1


In [7]:
regex = r'^(\bAnalysis\b|\bBackdoor\b|\bShellcode\b|\bWorms\b).*$'
train_cats = train_cats.replace(to_replace=regex, value='Other', regex=True)
train_cats.describe()

,attack_cat
count,82332
unique,7
top,Normal
freq,37000


In [8]:
train_labels = pd.get_dummies(train_cats)
train_labels.head()

,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Other,attack_cat_Reconnaissance
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0


In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_samples = scaler.fit_transform((train_samples))
scaled_labels = np.array(train_labels)

In [9]:
scaler = StandardScaler()
scaled_samples = scaler.fit_transform(train_samples)
scaled_labels = np.array(train_labels)
print scaled_samples.shape
print scaled_labels.shape

(82332, 17)
(82332, 7)


/home/jozef/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jozef/anaconda2/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### DNN model - 12, 32, 10
learning_rate = 0.001

epochs = 100

batch_size = 50

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(32, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(Dense(8, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 50

model = create_model()
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                416       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                330       
Total params: 746
Trainable params: 746
Non-trainable params: 0
_________________________________________________________________
Train on 65865 samples, validate on 16467 samples
Epoch 1/100
 - 2s - loss: 1.1616 - acc: 0.5964 - val_loss: 0.9963 - val_acc: 0.6094
Epoch 2/100
 - 2s - loss: 0.9598 - acc: 0.6643 - val_loss: 1.1065 - val_acc: 0.5552
Epoch 3/100
 - 2s - loss: 0.9271 - acc: 0.6745 - val_loss: 1.0396 - val_acc: 0.6444
Epoch 4/100
 - 2s - loss: 0.9028 - acc: 0.6839 - val_loss: 1.1127 - val_acc: 0.5430
Epoch 5/100
 - 2s - lo

### DNN model - 12, 160, 10
learning_rate = 0.001

epochs = 100

batch_size = 50

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(160, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(Dense(8, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(10, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

### DNN model - 17, 32, 7
epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(32, input_dim=17, activation='relu'))
    #model.add(Dropout(0.2))
    #model.add(Dense(8, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 50

estimator = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=2)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, scaled_samples, scaled_labels, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 2s - loss: 0.9167 - acc: 0.6884
Epoch 2/100
 - 2s - loss: 0.7303 - acc: 0.7421
Epoch 3/100
 - 2s - loss: 0.6967 - acc: 0.7500
Epoch 4/100
 - 1s - loss: 0.6787 - acc: 0.7557
Epoch 5/100
 - 1s - loss: 0.6650 - acc: 0.7623
Epoch 6/100
 - 1s - loss: 0.6547 - acc: 0.7666
Epoch 7/100
 - 1s - loss: 0.6467 - acc: 0.7671
Epoch 8/100
 - 1s - loss: 0.6401 - acc: 0.7707
Epoch 9/100
 - 1s - loss: 0.6340 - acc: 0.7713
Epoch 10/100
 - 1s - loss: 0.6284 - acc: 0.7744
Epoch 11/100
 - 1s - loss: 0.6245 - acc: 0.7741
Epoch 12/100
 - 1s - loss: 0.6203 - acc: 0.7763
Epoch 13/100
 - 1s - loss: 0.6168 - acc: 0.7776
Epoch 14/100
 - 1s - loss: 0.6137 - acc: 0.7771
Epoch 15/100
 - 1s - loss: 0.6102 - acc: 0.7789
Epoch 16/100
 - 1s - loss: 0.6079 - acc: 0.7795
Epoch 17/100
 - 1s - loss: 0.6055 - acc: 0.7809
Epoch 18/100
 - 1s - loss: 0.6026 - acc: 0.7807
Epoch 19/100
 - 1s - loss

### DNN model - 17, 128, 64, 32, 7
epochs = 100

batch_size = 20

dropout = 0.2

In [ ]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(128, input_dim=17, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

estimator = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, scaled_samples, scaled_labels, cv=kfold, n_jobs=-1, verbose=2)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


# Only 1 category and normal traffic

In [0]:
train_cats = data[['attack_cat']]
train_cats['attack_cat'].value_counts()

Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: attack_cat, dtype: int64

## Exploits & Normal traffic

In [0]:
exploits = data.loc[(data['attack_cat'] == 'Normal') | (data['attack_cat'] == 'Exploits')]
exploits[['attack_cat']].describe()

,attack_cat
count,48132
unique,2
top,Normal
freq,37000


In [0]:
train_others = exploits[['spkts','dpkts','sbytes','dbytes','ct_srv_dst','ct_dst_ltm']]
train_protos = exploits[['proto']]
train_labels = exploits[['label']]

In [0]:
regex = r'^(?!(\barp\b|\bigmp\b|\bospf\b|\btcp\b|\budp\b)).*$'
train_protos = train_protos.replace(to_replace=regex, value='other', regex=True)
train_protos.describe()

,proto
count,48132
unique,6
top,tcp
freq,35602


In [0]:
train_dummies = pd.get_dummies(train_protos)
train_samples = pd.concat([train_others, train_dummies], axis=1)
train_samples.head()

,spkts,dpkts,sbytes,dbytes,ct_srv_dst,ct_dst_ltm,proto_arp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
0,2,0,496,0,2,1,0,0,0,0,0,1
1,2,0,1762,0,2,1,0,0,0,0,0,1
2,2,0,1068,0,3,1,0,0,0,0,0,1
3,2,0,900,0,3,2,0,0,0,0,0,1
4,2,0,2126,0,3,2,0,0,0,0,0,1


In [0]:
scaler = StandardScaler()
scaled_samples = scaler.fit_transform((train_samples))
scaled_labels = np.array(train_labels)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python2.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
print scaled_samples.shape
print scaled_labels.shape

(48132, 12)
(48132, 1)


### DNN model - 12, 16, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(Dense(8, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                208       
_________________________________________________________________
dropout_7 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________
Train on 38505 samples, validate on 9627 samples
Epoch 1/100
 - 3s - loss: 0.3707 - acc: 0.8366 - val_loss: 0.2846 - val_acc: 0.8888
Epoch 2/100
 - 2s - loss: 0.2709 - acc: 0.8972 - val_loss: 0.2773 - val_acc: 0.8902
Epoch 3/100
 - 2s - loss: 0.2639 - acc: 0.9018 - val_loss: 0.2553 - val_acc: 0.8939
Epoch 4/100
 - 2s - loss: 0.2596 - acc: 0.9036 - val_loss: 0.2819 - val_acc: 0.8901
Epoch 5/100
 - 2s - los

### DNN model - 12, 16, 8, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                208       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 353
Trainable params: 353
Non-trainable params: 0
_______________________

In [0]:
model.save('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/DS/DNN_model_exploits.h5')

### K-fold cross validation

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

estimator = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose = 2)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, scaled_samples, scaled_labels, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
 - 4s - loss: 0.2870 - acc: 0.8933
Epoch 2/100
 - 3s - loss: 0.2426 - acc: 0.9140
Epoch 3/100
 - 3s - loss: 0.2272 - acc: 0.9194
Epoch 4/100
 - 3s - loss: 0.2150 - acc: 0.9223
Epoch 5/100
 - 3s - loss: 0.2057 - acc: 0.9242
Epoch 6/100
 - 3s - loss: 0.1944 - acc: 0.9281
Epoch 7/100
 - 3s - loss: 0.1802 - acc: 0.9341
Epoch 8/100
 - 3s - loss: 0.1665 - acc: 0.9418
Epoch 9/100
 - 3s - loss: 0.1575 - acc: 0.9473
Epoch 10/100
 - 3s - loss: 0.1498 - acc: 0.9518
Epoch 11/100
 - 3s - loss: 0.1432 - acc: 0.9547
Epoch 12/100
 - 3s - loss: 0.1390 - acc: 0.9570
Epoch 13/100
 - 3s - loss: 0.1375 - acc: 0.9567
Epoch 14/100
 - 3s - loss: 0.1379 - acc: 0.9572
Epoch 15/100
 - 3s - loss: 0.1328 - acc: 0.9587
Epoch 16/100
 - 3s - loss: 0.1307 - acc: 0.9599
Epoch 17/100
 - 3s - loss: 0.1286 - acc: 0.9606
Epoch 18/100
 - 3s - loss: 0.1266 - acc: 0.9627
Epoch 19/100
 - 3s - loss: 0.1218 - acc: 0.9631
Epoch 20/100
 - 3s - loss: 0.1206 - acc: 0.9636
Epoch 21/100
 - 3s - loss: 0.1186 - acc: 0.9648
E

## Generic & Normal traffic

In [0]:
generic = data.loc[(data['attack_cat'] == 'Normal') | (data['attack_cat'] == 'Generic')]
generic[['attack_cat']].describe()

,attack_cat
count,55871
unique,2
top,Normal
freq,37000


In [0]:
train_others = generic[['spkts','dpkts','sbytes','dbytes','ct_srv_dst','ct_dst_ltm']]
train_protos = generic[['proto']]
train_labels = generic[['label']]

In [0]:
regex = r'^(?!(\barp\b|\bigmp\b|\bospf\b|\btcp\b|\budp\b)).*$'
train_protos = train_protos.replace(to_replace=regex, value='other', regex=True)
train_protos.describe()

,proto
count,55871
unique,6
top,tcp
freq,28368


In [0]:
train_dummies = pd.get_dummies(train_protos)
train_samples = pd.concat([train_others, train_dummies], axis=1)
train_samples.head()

,spkts,dpkts,sbytes,dbytes,ct_srv_dst,ct_dst_ltm,proto_arp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
0,2,0,496,0,2,1,0,0,0,0,0,1
1,2,0,1762,0,2,1,0,0,0,0,0,1
2,2,0,1068,0,3,1,0,0,0,0,0,1
3,2,0,900,0,3,2,0,0,0,0,0,1
4,2,0,2126,0,3,2,0,0,0,0,0,1


In [0]:
scaler = StandardScaler()
scaled_samples = scaler.fit_transform((train_samples))
scaled_labels = np.array(train_labels)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python2.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
print scaled_samples.shape
print scaled_labels.shape

(55871, 12)
(55871, 1)


### DNN model - 12, 16, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 16)                208       
_________________________________________________________________
dropout_10 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 17        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________
Train on 44696 samples, validate on 11175 samples
Epoch 1/100
 - 3s - loss: 0.2825 - acc: 0.8931 - val_loss: 0.3101 - val_acc: 0.9079
Epoch 2/100
 - 3s - loss: 0.2334 - acc: 0.9176 - val_loss: 0.3043 - val_acc: 0.9184
Epoch 3/100
 - 3s - loss: 0.2252 - acc: 0.9223 - val_loss: 0.3046 - val_acc: 0.9233
Epoch 4/100
 - 3s - loss: 0.2191 - acc: 0.9243 - val_loss: 0.2755 - val_acc: 0.9188
Epoch 5/100
 - 3s - lo

### DNN model - 12, 16, 8, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                208       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 353
Trainable params: 353
Non-trainable params: 0
_______________________

In [0]:
model.save('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/DS/DNN_model_generic.h5')

### K-fold cross validation

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

estimator = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose = 2)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, scaled_samples, scaled_labels, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
 - 5s - loss: 0.2870 - acc: 0.8933
Epoch 2/100
 - 3s - loss: 0.2427 - acc: 0.9141
Epoch 3/100
 - 3s - loss: 0.2271 - acc: 0.9184
Epoch 4/100
 - 3s - loss: 0.2155 - acc: 0.9218
Epoch 5/100
 - 3s - loss: 0.2058 - acc: 0.9239
Epoch 6/100
 - 3s - loss: 0.1939 - acc: 0.9285
Epoch 7/100
 - 3s - loss: 0.1802 - acc: 0.9342
Epoch 8/100
 - 3s - loss: 0.1663 - acc: 0.9420
Epoch 9/100
 - 3s - loss: 0.1569 - acc: 0.9475
Epoch 10/100
 - 3s - loss: 0.1490 - acc: 0.9524
Epoch 11/100
 - 3s - loss: 0.1423 - acc: 0.9552
Epoch 12/100
 - 3s - loss: 0.1383 - acc: 0.9573
Epoch 13/100
 - 3s - loss: 0.1367 - acc: 0.9567
Epoch 14/100
 - 3s - loss: 0.1375 - acc: 0.9571
Epoch 15/100
 - 3s - loss: 0.1318 - acc: 0.9590
Epoch 16/100
 - 3s - loss: 0.1303 - acc: 0.9595
Epoch 17/100
 - 3s - loss: 0.1296 - acc: 0.9600
Epoch 18/100
 - 3s - loss: 0.1278 - acc: 0.9619
Epoch 19/100
 - 3s - loss: 0.1231 - acc: 0.9623
Epoch 20/100
 - 3s - loss: 0.1223 - acc: 0.9622
Epoch 21/100
 - 3s - loss: 0.1223 - acc: 0.9626
E

## Reconnaissance & Normal traffic

In [0]:
recon = data.loc[(data['attack_cat'] == 'Normal') | (data['attack_cat'] == 'Reconnaissance')]
recon[['attack_cat']].describe()

,attack_cat
count,40496
unique,2
top,Normal
freq,37000


In [0]:
train_others = recon[['spkts','dpkts','sbytes','dbytes','ct_srv_dst','ct_dst_ltm']]
train_protos = recon[['proto']]
train_labels = recon[['label']]

In [0]:
regex = r'^(?!(\barp\b|\bigmp\b|\bospf\b|\btcp\b|\budp\b)).*$'
train_protos = train_protos.replace(to_replace=regex, value='other', regex=True)
train_protos.describe()

,proto
count,40496
unique,6
top,tcp
freq,29713


In [0]:
train_dummies = pd.get_dummies(train_protos)
train_samples = pd.concat([train_others, train_dummies], axis=1)
train_samples.head()

,spkts,dpkts,sbytes,dbytes,ct_srv_dst,ct_dst_ltm,proto_arp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
0,2,0,496,0,2,1,0,0,0,0,0,1
1,2,0,1762,0,2,1,0,0,0,0,0,1
2,2,0,1068,0,3,1,0,0,0,0,0,1
3,2,0,900,0,3,2,0,0,0,0,0,1
4,2,0,2126,0,3,2,0,0,0,0,0,1


In [0]:
scaler = StandardScaler()
scaler.fit(train_samples)
scaled_samples = scaler.transform(train_samples)
scaled_labels = np.array(train_labels)
print scaled_samples.shape
print scaled_labels.shape

(40496, 12)
(40496, 1)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


### DNN model - 12, 16, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                208       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________
Train on 32396 samples, validate on 8100 samples
Epoch 1/100
 - 2s - loss: 0.3024 - acc: 0.8803 - val_loss: 0.1258 - val_acc: 1.0000
Epoch 2/100
 - 2s - loss: 0.2085 - acc: 0.9088 - val_loss: 0.1015 - val_acc: 1.0000
Epoch 3/100
 - 2s - loss: 0.1956 - acc: 0.9160 - val_loss: 0.1007 - val_acc: 0.9983
Epoch 4/100
 - 2s - loss: 0.1905 - acc: 0.9230 - val_loss: 0.0953 - val_acc: 0.9902
Epoch 5/100
 - 2s - los

### DNN model - 12, 16, 8, 1
learning_rate = 0.001

epochs = 100

batch_size = 20

dropout = 0.2

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

model = create_model()
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(scaled_samples, scaled_labels, validation_split=0.2, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                208       
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dropout_7 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________
Train on 32396 samples, validate on 8100 samples
Epoch 1/100
 - 3s - loss: 0.2937 - acc: 0.8864 - val_loss: 0.1050 - val_acc: 1.0000
Epoch 2/100

In [0]:
model.save('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/DS/DNN_model_reconnaissance.h5')
joblib.dump(scaler, '/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/DS/DNN_model_reconnaissance_scaler.sav')

['/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomovy\xcc\x81 projekt 3/DS/DNN_model_reconnaissance_scaler.sav']

### K-fold cross validation

In [0]:
def create_model():
    #create a model
    model = Sequential()
    model.add(Dense(16, input_dim=12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model
  
seed = 7
np.random.seed(seed)
epochs = 100
batch_size = 20

estimator = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose = 2)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, scaled_samples, scaled_labels, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))